In [ ]:
import yfinance as yf
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from dash.dependencies import Input, Output

# Ações para análise
acoes = ['AAPL', 'MSFT']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles das Ações AAPL e MSFT"),
    dcc.Graph(id='candlestick-plot')
])

@app.callback(
    Output('candlestick-plot', 'figure'),
    [Input('candlestick-plot', 'relayoutData')]
)
def update_candlestick_plot(relayout_data):
    return {
        'data': [
            go.Candlestick(
                x=dados_acoes.index,
                open=dados_acoes[acao]['Open'],
                high=dados_acoes[acao]['High'],
                low=dados_acoes[acao]['Low'],
                close=dados_acoes[acao]['Adj Close'],
                name=acao
            )
            for acao in acoes
        ],
        'layout': {
            'title': 'Gráfico de Candles das Ações AAPL e MSFT',
            'xaxis': {'title': 'Data'},
            'yaxis': {'title': 'Preço de Fechamento'}
        }
    }

if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import yfinance as yf
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask com Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div([
    html.H1("Gráfico de Candles da Ação do Facebook (FB)"),
    dcc.Graph(
        id='candlestick-plot',
        figure={
            'data': [
                go.Candlestick(
                    x=acao_facebook.index,
                    open=acao_facebook['Open'],
                    high=acao_facebook['High'],
                    low=acao_facebook['Low'],
                    close=acao_facebook['Close'],
                    name='FB'
                )
            ],
            'layout': {
                'title': 'Gráfico de Candles da Ação do Facebook (FB)',
                'xaxis': {'title': 'Data'},
                'yaxis': {'title': 'Preço'},
                'yaxis2': {'title': 'Volume', 'overlaying': 'y', 'side': 'right'}
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = go.Figure(data=[go.Candlestick(x=acao_facebook.index,
                                        open=acao_facebook['Open'],
                                        high=acao_facebook['High'],
                                        low=acao_facebook['Low'],
                                        close=acao_facebook['Close'])])

    fig.update_layout(title='Gráfico de Candles da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      yaxis2=dict(overlaying='y', side='right'))

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import yfinance as yf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from flask import Flask, render_template

# Obtendo os dados históricos da ação do Facebook
acao_facebook = yf.download('FB', start='2022-01-01', end='2023-01-01')

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o gráfico de candles da ação do Facebook
@app.route('/')
def plot_candlestick():
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)

    # Gráfico de candles
    fig.add_trace(go.Candlestick(x=acao_facebook.index,
                                 open=acao_facebook['Open'],
                                 high=acao_facebook['High'],
                                 low=acao_facebook['Low'],
                                 close=acao_facebook['Close'],
                                 name='FB'), row=1, col=1)

    # Gráfico de volume
    fig.add_trace(go.Bar(x=acao_facebook.index,
                         y=acao_facebook['Volume'],
                         name='Volume'), row=2, col=1)

    fig.update_layout(title='Gráfico de Candles e Volume da Ação do Facebook (FB)',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis2_title='Volume',
                      showlegend=False)

    return render_template('candlestick_plot.html', plot=fig.to_html())

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import yfinance as yf
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.models.widgets import Select
from datetime import datetime

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

# Criando a fonte de dados para o gráfico
source = ColumnDataSource(dados_acoes['Close'])

# Criando o gráfico de linha para cada ação
p = figure(x_axis_type='datetime', title='Preço de Fechamento das Ações', width=800, height=400)
for acao in acoes:
    p.line(x='Date', y=acao, source=source, line_width=2, legend_label=acao)

# Adicionando as ferramentas de interação
hover = HoverTool()
hover.tooltips = [("Data", "@Date{%F}"), ("Preço de Fechamento", "$y")]
hover.formatters = {'@Date': 'datetime'}
p.add_tools(hover)

p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

# Criando o menu dropdown para selecionar a ação
menu = Select(options=acoes, value='AAPL', title='Selecione uma Ação')

# Função para atualizar o gráfico ao selecionar uma ação no menu dropdown
def update_plot(attr, old, new):
    selected_acao = menu.value
    p.title.text = f"Preço de Fechamento da Ação {selected_acao}"
    source.data = ColumnDataSource(dados_acoes[selected_acao]).data

menu.on_change('value', update_plot)

# Organizando o layout do dashboard
layout = column(menu, p)

# Configurando o documento do Bokeh
curdoc().add_root(layout)


In [16]:
import yfinance as yf
import plotly.graph_objects as go

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2023-01-01', end='2023-06-01')

# Função para criar o gráfico de candles
def criar_grafico_candles(acao):
    candlestick = go.Candlestick(x=dados_acoes.index,
                                 open=dados_acoes[acao]['Open'][0],
                                 high=dados_acoes[acao]['High'][1],
                                 low=dados_acoes[acao]['Low'][2],
                                 close=dados_acoes[acao]['Close'][3],
                                 name=acao)

    return candlestick

# Criando lista com os gráficos de candles para cada ação
graficos_candles = [criar_grafico_candles(acao) for acao in acoes]

# Layout do gráfico
layout = go.Layout(title='Gráficos de Candles das Ações',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com os gráficos de candles
figura = go.Figure(data=graficos_candles, layout=layout)

# Exibindo o gráfico
figura.show()


[*********************100%***********************]  4 of 4 completed


KeyError: 'AAPL'

In [15]:
import yfinance as yf
import plotly.graph_objects as go

# Lista de ações para análise
acoes = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Obtendo os dados históricos das ações
dados_acoes = yf.download(acoes, start='2022-01-01', end='2023-01-01')

dados_acoes

[*********************100%***********************]  4 of 4 completed


Adj Close                                           Close  \
                  AAPL        AMZN       GOOGL        MSFT        AAPL   
Date                                                                     
2022-01-03  180.434296  170.404495  144.991501  330.092194  182.009995   
2022-01-04  178.144302  167.522003  144.399506  324.432159  179.699997   
2022-01-05  173.405685  164.356995  137.774994  311.977905  174.919998   
2022-01-06  170.510971  163.253998  137.747498  309.512634  172.000000   
2022-01-07  170.679474  162.554001  137.016998  309.670441  172.169998   
...                ...         ...         ...         ...         ...   
2022-12-23  131.477127   85.250000   89.230003  237.614105  131.860001   
2022-12-27  129.652435   83.040001   87.389999  235.852386  130.029999   
2022-12-28  125.674019   81.820000   86.019997  233.433731  126.040001   
2022-12-29  129.233658   84.180000   88.449997  239.883438  129.610001   
2022-12-30  129.552719   84.000000   88.230003  238.699020  129.929993   

                                                      High              ...  \
                  AMZN       GOOGL        MSFT        AAPL        AMZN  ...   
Date                                                                    ...   
2022-01-03  170.404495  144.991501  334.750000  182.880005  170.703506  ...   
2022-01-04  167.522003  144.399506  329.010010  182.940002  171.399994  ...   
2022-01-05  164.356995  137.774994  316.380005  180.169998  167.126495  ...   
2022-01-06  163.253998  137.747498  313.880005  175.300003  164.800003  ...   
2022-01-07  162.554001  137.016998  314.040009  174.139999  165.243500  ...   
...                ...         ...         ...         ...         ...  ...   
2022-12-23   85.250000   89.230003  238.729996  132.419998   85.779999  ...   
2022-12-27   83.040001   87.389999  236.960007  131.410004   85.349998  ...   
2022-12-28   81.820000   86.019997  234.529999  131.029999   83.480003  ...   
2022-12-29   84.180000   88.449997  241.009995  130.479996   84.550003  ...   
2022-12-30   84.000000   88.230003  239.820007  129.949997   84.050003  ...   

                   Low                    Open                          \
                 GOOGL        MSFT        AAPL        AMZN       GOOGL   
Date                                                                     
2022-01-03  143.712997  329.779999  177.830002  167.550003  145.054993   
2022-01-04  143.716507  326.119995  182.630005  170.438004  145.395996   
2022-01-05  137.688004  315.980011  179.610001  166.882996  144.419998   
2022-01-06  136.558502  311.489990  172.699997  163.450500  136.998505   
2022-01-07  135.766495  310.089996  172.889999  163.839005  138.145493   
...                ...         ...         ...         ...         ...   
2022-12-23   87.070000  233.940002  130.919998   83.250000   87.110001   
2022-12-27   87.010002  235.830002  131.380005   84.970001   88.800003   
2022-12-28   85.940002  234.169998  129.669998   82.800003   86.980003   
2022-12-29   86.610001  235.649994  127.989998   82.870003   86.620003   
2022-12-30   86.570000  236.660004  128.410004   83.120003   86.980003   

                           Volume                                
                  MSFT       AAPL      AMZN     GOOGL      MSFT  
Date                                                             
2022-01-03  335.350006  104487900  63520000  28646000  28865100  
2022-01-04  334.829987   99310400  70726000  28400000  32674300  
2022-01-05  325.859985   94537600  64302000  54618000  40054300  
2022-01-06  313.149994   96904000  51958000  37348000  39646100  
2022-01-07  314.149994   86709100  46606000  29760000  32720000  
...                ...        ...       ...       ...       ...  
2022-12-23  236.110001   63814900  57433700  23003000  21207000  
2022-12-27  238.699997   69007800  57284000  20097300  16688600  
2022-12-28  236.889999   85438400  58228600  19523200  17457100  
2022-12-29  235.649994   75703700  54995

In [17]:
import yfinance as yf
import plotly.graph_objects as go

# Ação para análise
acao = 'AAPL'  # Altere para a ação desejada

# Obtendo os dados históricos da ação
dados_acao = yf.download(acao, start='2022-01-01', end='2023-01-01')

# Criando o gráfico de candles
candlestick = go.Candlestick(x=dados_acao.index,
                             open=dados_acao['Open'],
                             high=dados_acao['High'],
                             low=dados_acao['Low'],
                             close=dados_acao['Close'],
                             name=acao)

# Layout do gráfico
layout = go.Layout(title=f'Gráfico de Candles da Ação {acao}',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com o gráfico de candles
figura = go.Figure(data=[candlestick], layout=layout)

# Exibindo o gráfico
figura.show()


[*********************100%***********************]  1 of 1 completed


In [18]:
import yfinance as yf
import plotly.graph_objects as go
from flask import Flask, render_template

# Ação para análise
acao = 'AAPL'  # Altere para a ação desejada

# Obtendo os dados históricos da ação
dados_acao = yf.download(acao, start='2022-01-01', end='2023-01-01')

# Criando o gráfico de candles
candlestick = go.Candlestick(x=dados_acao.index,
                             open=dados_acao['Open'],
                             high=dados_acao['High'],
                             low=dados_acao['Low'],
                             close=dados_acao['Close'],
                             name=acao)

# Layout do gráfico
layout = go.Layout(title=f'Gráfico de Candles da Ação {acao}',
                   xaxis=dict(title='Data'),
                   yaxis=dict(title='Preço'))

# Criando a figura com o gráfico de candles
figura = go.Figure(data=[candlestick], layout=layout)

# Criando o aplicativo Flask
app = Flask(__name__)

# Rota para renderizar o template com o gráfico
@app.route('/')
def renderizar_template():
    # Converte a figura para HTML e passa para o template
    grafico_html = figura.to_html(full_html=False, include_plotlyjs='cdn')
    return render_template('grafico.html', grafico_html=grafico_html)

if __name__ == '__main__':
    app.run(debug=True)


[*********************100%***********************]  1 of 1 completed
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\Caíque Miranda\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

